<a href="https://colab.research.google.com/github/Dann-El/AP_MAq/blob/main/Atividade1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import requests
def obter_link_gd(link):
  id_arquivo = link.split('/')[-2]
  api_url = f'https://drive.google.com/uc?id={id_arquivo}'
  retorno = requests.get(api_url)

  if retorno.status_code == 200:
   link2 = retorno.url
   return link2

  else:
    print("Falha")
    return None


In [ ]:
from io import StringIO

link_csv = 'https://drive.google.com/file/d/1AT0EwYB8VfW8RU-QUOktPzRG8e1NQZqu/view?usp=sharing'
link_final = obter_link_gd(link_csv)

if link_final:
  try:
    df = pd.read_csv(link_final)
  except pd.erros.ParseError as e:
    print(f'falha: {e}')
else:
  print('falha total')


In [ ]:
df.head(5)
df.shape
df.isnull().sum()

Age                   0
Gender                0
Income                0
Education             0
Marital Status        0
Number of Children    0
Home Ownership        0
Credit Score          0
dtype: int64

In [ ]:
print(df.head(5),"\n",df.columns)

   Age  Gender  Income            Education Marital Status  \
0   25  Female   50000    Bachelor's Degree         Single   
1   30    Male  100000      Master's Degree        Married   
2   35  Female   75000            Doctorate        Married   
3   40    Male  125000  High School Diploma         Single   
4   45  Female  100000    Bachelor's Degree        Married   

   Number of Children Home Ownership Credit Score  
0                   0         Rented         High  
1                   2          Owned         High  
2                   1          Owned         High  
3                   0          Owned         High  
4                   3          Owned         High   
 Index(['Age', 'Gender', 'Income', 'Education', 'Marital Status',
       'Number of Children', 'Home Ownership', 'Credit Score'],
      dtype='object')


In [ ]:
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

df['Marital Status'] = df['Marital Status'].map({'Single': 2, 'Married': 1})

mapa = {'Rented': 1, 'Owned': 2}
df["Home Ownership"] = df["Home Ownership"].map(mapa)

df["Credit Score"] = df["Credit Score"].map({'High': 3, 'Average': 2, 'Low':1 },)

df.head(2)

,Age,Gender,Income,Education,Marital Status,Number of Children,Home Ownership,Credit Score
0,25,0,50000,Bachelor's Degree,2,0,1,3
1,30,1,100000,Master's Degree,1,2,2,3


In [ ]:
df.Education.unique()

array(["Bachelor's Degree", "Master's Degree", 'Doctorate',
       'High School Diploma', "Associate's Degree"], dtype=object)

In [ ]:
novas_colunas = pd.get_dummies(df.Education, drop_first = True)
df[["Master's Degree", 'Doctorate',
       'High School Diploma', "Associate's Degree"]] = novas_colunas

In [ ]:
X = df.drop('Education', axis = 1)
y = X["Credit Score"]

In [ ]:
X.head(3)
X.columns

Index(['Age', 'Gender', 'Income', 'Marital Status', 'Number of Children',
       'Home Ownership', 'Credit Score', 'Master's Degree', 'Doctorate',
       'High School Diploma', 'Associate's Degree'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(X.shape,'\n', X_train.shape, X_test.shape)

(164, 11) 
 (123, 11) (41, 11)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit_transform(X_train)
scaler.transform(X_test)
y_test.head(5)

97     3
110    3
80     2
106    2
150    3
Name: Credit Score, dtype: int64

In [ ]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
y_test.head(10)

97     3
110    3
80     2
106    2
150    3
124    3
83     3
86     2
16     3
160    2
Name: Credit Score, dtype: int64

In [ ]:
pred[:10]

array([3, 3, 2, 2, 3, 3, 3, 2, 3, 2])

In [ ]:
from sklearn.metrics import precision_score
precisao_micro = precision_score(y_test, pred, average = 'macro')
precisao_micro

0.9209876543209877

In [ ]:
X['Credit Score'].value_counts()

3    113
2     36
1     15
Name: Credit Score, dtype: int64

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(4)
folds = skf.split(X, y)
folds_list = list(folds)

In [ ]:
len(folds_list)
len(folds_list[3][0])

123

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline(steps = [('scaler', StandardScaler()), ('knn', neighbors.KNeighborsClassifier(3))])

In [ ]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(pipe, X, y, cv = folds_list, scoring = ('precision_micro'))
score

array([0.97560976, 1.        , 1.        , 0.92682927])

In [ ]:
score.std()

0.029871826131502154

In [ ]:
from sklearn.model_selection import GridSearchCV
parametros = {'knn__n_neighbors' : [2, 3, 4]}
search = GridSearchCV(pipe, parametros, cv = folds_list, scoring = 'precision_micro')
search.fit(X, y)

GridSearchCV(cv=[(array([ 40,  41,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136,...
       118, 119, 120, 121, 122, 125]),
                  array([117, 123, 124, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163]))],
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('knn',
                                        KNeighborsClassifier(n_neighbors=3))]),
             param_grid={'knn__n_neighbors': [2, 3, 4]},
             scoring='precision_micro')

In [ ]:
search.best_score_
"0.975609756097561"

'0.975609756097561'

In [ ]:
search.best_params_

{'knn__n_neighbors': 2}